In [4]:
import json
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.utils import pad_sequences
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import numpy as np
features_d2 = []
labels_d2 = []
# Open file for reading
with open('domain2_train.json', 'r') as f:
    for line in f:
        # Parse the JSON line into a Python dictionary
        obj = json.loads(line)
        features_d2.append(obj['text'])
        labels_d2.append(obj['label'])            

In [56]:

def to_bow(sequences, dimension=5000):  # 5000 considering 0 to 4999
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for word_index in sequence:
            results[i, word_index] += 1  # Increment by 1 for each occurrence
    return results

Bow_features = to_bow(features_d2)

In [72]:
def dummy_fun(doc):
    return doc

tfidf = TfidfVectorizer(
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None)  

# Fit and transform the data to text
features_tfidf = tfidf.fit_transform(features_d2)
label_d2 = np.array(labels_d2)

In [87]:
X_train, X_test, y_train, y_test = train_test_split(Bow_features, labels_d2, test_size=0.2, random_state=42)
from sklearn.linear_model import SGDClassifier

# Initialize and fit the model
clf = SGDClassifier(loss="hinge", class_weight={0: 1, 1: 6}, alpha=0.001, penalty='elasticnet')  # 'hinge' loss gives a linear SVM
clf.fit(X_train, y_train)

# Get predictions
y_pred = clf.predict(X_test)

# Generate the classification report
report = classification_report(y_test, y_pred)

print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.87      0.87      2544
           1       0.28      0.29      0.29       436

    accuracy                           0.79      2980
   macro avg       0.58      0.58      0.58      2980
weighted avg       0.79      0.79      0.79      2980



In [99]:




import lightgbm as lgb
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(Bow_features, labels_d2, test_size=0.2, random_state=42)

d_train = lgb.Dataset(X_train, label=y_train)  
d_test = lgb.Dataset(X_test, label=y_test)  

best_params = {
    'colsample_bytree': 0.72,
    'learning_rate': 0.01642,
    'max_depth': 23,
    'min_child_samples': 60,
    'num_leaves': 50,
    'n_estimators': 470,
    'scale_pos_weight': 8,
    'subsample': 0.93446,
    'reg_alpha': 0.30421732656825395

}

clf = lgb.train(best_params, d_train,verbose_eval=1, num_boost_round=25000,  valid_sets=[d_test], early_stopping_rounds=1000)




from sklearn.metrics import classification_report

# Generate predicted classes
y_val_pred_class = [1 if x >= 0.5 else 0 for x in clf.predict(X_test)]

# Generate the classification report
report = classification_report(y_test, y_val_pred_class)

c:\Users\chenj\anaconda3\envs\CV\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\Users\chenj\anaconda3\envs\CV\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\chenj\anaconda3\envs\CV\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.034726 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11761
[LightGBM] [Info] Number of data points in the train set: 11920, number of used features: 2171
[LightGBM] [Info] Start training from score 0.143792
[1]	valid_0's l2: 0.124517
Training until validation scores don't improve for 1000 rounds
[2]	valid_0's l2: 0.12414
[3]	valid_0's l2: 0.123725
[4]	valid_0's l2: 0.123492
[5]	valid_0's l2: 0.123176
[6]	valid_0's l2: 0.122795
[7]	valid_0's l2: 0.122434
[8]	valid_0's l2: 0.122224
[9]	valid_0's l2: 0.121909
[10]	valid_0's l2: 0.121786
[11]	valid_0's l2: 0.121468
[12]	valid_0's l2: 0.121276
[13]	valid_0's l2: 0.120968
[14]	valid_0's l2: 0.120688
[15]	valid_0's l2: 0.120409
[16]	valid_0's l2: 0.120144
[17]	valid_0's l2: 0.119933
[18]	valid_0's l2: 0.119673
[19]	valid_0's l2: 0.119403
[20

In [100]:
print(report)

              precision    recall  f1-score   support

           0       0.86      0.99      0.92      2544
           1       0.63      0.10      0.17       436

    accuracy                           0.86      2980
   macro avg       0.75      0.54      0.55      2980
weighted avg       0.83      0.86      0.81      2980



In [7]:
def dummy_fun(doc):
    return doc

tfidf = TfidfVectorizer(
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None)  

# Fit and transform the data to text
features_tfidf = tfidf.fit_transform(features_d2)
label_d2 = np.array(labels_d2)

In [99]:

import pickle

# Save the trained vectorizer
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)


In [8]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features_tfidf, labels_d2, test_size=0.2, random_state=42)


In [4]:
import lightgbm as lgb

In [140]:
from skopt import BayesSearchCV
import lightgbm as lgb

from sklearn.metrics import accuracy_score


# Define the simplified hyperparameter configuration space
param_space = {
    'num_leaves': (100,120),
    'max_depth': (80,110),
    'min_child_samples': (140, 160),
    'subsample': (0.7, 1.0),
    'colsample_bytree': (0.6, 1.0),
    'scale_pos_weight': (5, 10),
    'learning_rate': (0.01, 0.1, 'log-uniform'),
    'n_estimators': (600, 1200),
    'reg_alpha': (0.0, 1.0),

}




opt = BayesSearchCV(
    estimator=lgb.LGBMClassifier(
        objective='binary',
        metric='binary_error',
        n_jobs=2,
        verbose= -1
    ),
    search_spaces=param_space,
    scoring='f1',
    cv=5,
    n_jobs=1,
    n_iter=30,
    verbose=100,
    refit=True,
)

opt.fit(Bow_features, labels_d2)



Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START colsample_bytree=0.783842418575133, learning_rate=0.014772439883158385, max_depth=80, min_child_samples=148, n_estimators=685, num_leaves=109, reg_alpha=0.008502122718325624, scale_pos_weight=10, subsample=0.7511734711732679
[CV 1/5; 1/1] END colsample_bytree=0.783842418575133, learning_rate=0.014772439883158385, max_depth=80, min_child_samples=148, n_estimators=685, num_leaves=109, reg_alpha=0.008502122718325624, scale_pos_weight=10, subsample=0.7511734711732679;, score=0.421 total time=   3.9s
[CV 2/5; 1/1] START colsample_bytree=0.783842418575133, learning_rate=0.014772439883158385, max_depth=80, min_child_samples=148, n_estimators=685, num_leaves=109, reg_alpha=0.008502122718325624, scale_pos_weight=10, subsample=0.7511734711732679
[CV 2/5; 1/1] END colsample_bytree=0.783842418575133, learning_rate=0.014772439883158385, max_depth=80, min_child_samples=148, n_estimators=685, num_leaves=109, reg_alpha=0.00

BayesSearchCV(cv=5,
              estimator=LGBMClassifier(metric='binary_error', n_jobs=2,
                                       objective='binary', verbose=-1),
              n_iter=30, scoring='f1',
              search_spaces={'colsample_bytree': (0.6, 1.0),
                             'learning_rate': (0.01, 0.1, 'log-uniform'),
                             'max_depth': (80, 110),
                             'min_child_samples': (140, 160),
                             'n_estimators': (600, 1200),
                             'num_leaves': (100, 120), 'reg_alpha': (0.0, 1.0),
                             'scale_pos_weight': (5, 10),
                             'subsample': (0.7, 1.0)},
              verbose=100)

In [141]:
opt.best_score_

0.4111488885340888

In [142]:
opt.best_params_

OrderedDict([('colsample_bytree', 0.6915100711795957),
             ('learning_rate', 0.010355565762216848),
             ('max_depth', 80),
             ('min_child_samples', 143),
             ('n_estimators', 901),
             ('num_leaves', 100),
             ('reg_alpha', 0.947914181540724),
             ('scale_pos_weight', 7),
             ('subsample', 0.7806775665122635)])

In [143]:
best_params = opt.best_params_




# Manually train a model using the best params
clf = lgb.LGBMClassifier(**best_params)
clf.fit(X_train, y_train)

# Generate predicted classes
y_val_pred_class = [1 if x >= 0.5 else 0 for x in clf.predict(X_test)]

# Generate the classification report
report = classification_report(y_test, y_val_pred_class)

#Save model
clf.booster_.save_model('domain2_model.txt')



In [144]:
print(report)

              precision    recall  f1-score   support

           0       0.92      0.80      0.86      2544
           1       0.34      0.59      0.43       436

    accuracy                           0.77      2980
   macro avg       0.63      0.70      0.64      2980
weighted avg       0.83      0.77      0.79      2980



In [61]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, f1_score
import lightgbm as lgb

# Create a LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)




best_params = {
    'colsample_bytree': 0.72,
    'learning_rate': 0.01642,
    'max_depth': 23,
    'min_child_samples': 60,
    'num_leaves': 50,
    'n_estimators': 470,
    'scale_pos_weight': 8,
    'subsample': 0.93446,
    'reg_alpha': 0.30421732656825395

}
bst = lgb.train(best_params, train_data, valid_sets=[test_data])

# Make predictions
y_pred = bst.predict(X_test)
# Convert to binary output
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]

# Evaluate the model
f1s = f1_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)
auc = roc_auc_score(y_test, y_pred)  # y_pred is the probability, not the binary output

print(f"f1s: {f1s}")
print(f"Precision: {precision}")
print(f"AUC: {auc}")

c:\Users\chenj\anaconda3\envs\CV\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.095362 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 162112
[LightGBM] [Info] Number of data points in the train set: 11920, number of used features: 2171
[LightGBM] [Info] Start training from score 0.143792
[1]	valid_0's l2: 0.12454
[2]	valid_0's l2: 0.124207
[3]	valid_0's l2: 0.123862
[4]	valid_0's l2: 0.123668
[5]	valid_0's l2: 0.12336
[6]	valid_0's l2: 0.123078
[7]	valid_0's l2: 0.122752
[8]	valid_0's l2: 0.122563
[9]	valid_0's l2: 0.122279
[10]	valid_0's l2: 0.122182
[11]	valid_0's l2: 0.121908
[12]	valid_0's l2: 0.121738
[13]	valid_0's l2: 0.12146
[14]	valid_0's l2: 0.121204
[15]	valid_0's l2: 0.120948
[16]	valid_0's l2: 0.120745
[17]	valid_0's l2: 0.120582
[18]	valid_0's l2: 0.120358
[19]	valid_0's l2: 0.120078
[20]	valid_0's l2: 0.11981
[21]	valid_0's l2: 0.119687
[22]	valid_0

In [ ]:
# Make predictions
y_pred = opt.predict(X_test)

# Evaluate the model
precision = precision_score(y_test, y_pred_binary)
print(f"Precision: {precision}")

Precision: 0.7142857142857143


In [60]:
from sklearn.metrics import recall_score
recall_score(y_test, y_pred_binary)

0.052752293577981654

In [ ]:
precision

1.0